In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [4]:
df['point_bins'] = pd.cut(
    df['total_points_per_game'],
    [0, 30, 35, 40, 45, float("inf")],
    right=False,
    labels=['< 30', '30 - 35', '35 - 40', '40 - 45', "> 45"]
)

In [6]:
df.head()

,date,team_1_player_1,team_1_player_2,team_2_player_1,team_2_player_2,points_team_1,points_team_2,venue,winner,margin,total_points_per_game,point_bins
0,12/27/2022,vinay,raghotham,ajay,aakarsh,15,21,isro match point,team_2,6,36,35 - 40
1,12/28/2022,vinay,raghotham,ajay,sameer,25,27,isro match point,team_2,2,52,> 45
2,12/28/2022,vinay,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40
3,12/28/2022,sameer,raghotham,vinay,srinidhi,14,21,isro match point,team_2,7,35,35 - 40
4,12/28/2022,sameer,raghotham,ajay,srinidhi,21,18,isro match point,team_1,3,39,35 - 40


In [10]:
players_list = list(np.unique(df[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].values))

In [15]:
def get_player_stats(player, df):
    player_matches = df[
        np.where(
            np.logical_or.reduce([df[i] == player for i in ["team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2"]]),
            True,
            False
        )
    ].copy()
    
    player_matches["belongs_to"] = np.where(
        np.logical_or(
            *[player_matches[i] == player for i in ["team_1_player_1", "team_1_player_2"]]
        ),
        'team_1',
        'team_2'
    )
    
    player_matches['player_team_points'] = np.where(
        player_matches["belongs_to"] == 'team_1',
        player_matches["points_team_1"],
        player_matches["points_team_2"]
    )
    
    player_matches['result'] = np.where(player_matches.belongs_to == player_matches.winner, "win", "loss")
    player_matches['is_win'] = np.where(player_matches.result == "win", 1, 0)
    
    return player_matches

In [80]:
leaderboard = []

for player in players_list:
    player_stats = get_player_stats(player, df)
    
    total_games, wins = player_stats.shape[0], player_stats['is_win'].sum()
    leaderboard.append({
        "player": player,
        "total_games": total_games,
        "wins": wins,
        "wins_pct": round(wins * 100 / total_games, 2),
        "form": ' '.join(player_stats['result'][-5:].apply(lambda x: x[0].upper()).to_list())
    })

leaderboard_df = pd.DataFrame(leaderboard).sort_values("wins_pct", ascending=False)

In [81]:
# pd.DataFrame(leaderboard).set_index("player").sort_values("wins_pct", ascending=False).to

In [90]:
go_table = go.Table(
    header=dict(
        values=leaderboard_df.columns, 
        align="left", 
        height=40, 
        fill_color="lightslategrey", 
        font=dict(color="white", size=16, family="Arial")
    ),
    cells=dict(
        values=[leaderboard_df[i] for i in leaderboard_df.columns], 
        align="left", 
        height=30,
        fill_color=[np.where(leaderboard_df['wins_pct'] == leaderboard_df['wins_pct'].max(), '#b5de2b', '#eceff1')]
    ),
    name="Leaderboard"
)
go.Figure(
    go_table
)

In [99]:
go_table = go.Table(
    header=dict(
        values=leaderboard_df.columns, 
        align="left", 
        height=40, 
        fill_color="#263238", 
        font=dict(color="white", size=16, family="Arial")
    ),
    cells=dict(
        values=[leaderboard_df[i] for i in leaderboard_df.columns], 
        align="left", 
        height=30,
        fill_color="#1f2e2e",
        font=dict(color=[np.where(leaderboard_df['wins_pct'] == leaderboard_df['wins_pct'].max(), '#8bc34a', 'white')]),
        line=dict(color="lightslategrey")
    ),
    name="Leaderboard"
)
go.Figure(
    go_table
)